In [1]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from tqdm import tqdm

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Dropout
from keras import backend as K

import plotly.graph_objects as go

In [2]:
list_name = 'data/Stock_List.csv'
stock_list = pd.read_csv(list_name)
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [3]:
sub = pd.read_csv('data/sample_submission.csv')
sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def NMSE_metric(y_true, y_pred):
    nmse = K.square(y_true - y_pred)/K.square(y_true + 1e-8)
    return nmse 

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

In [4]:
start_date = '20000101'
end_date = '20201231'


for code in tqdm(stock_list['종목코드']):

    df = fdr.DataReader(code, start = start_date, end = end_date)
    df.reset_index(inplace=True)
    df.set_axis(df['Date'], inplace=True)

    close_data = df['Close'].values
    close_data = close_data.reshape((-1,1))

    split_percent = 0.90
    split = int(split_percent*len(close_data))

    close_train = close_data[:split]
    close_test = close_data[split:]

    date_train = df['Date'][:split]
    date_test = df['Date'][split:]

    look_back = 15

    train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
    test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

    model = Sequential()
    
    model.add(LSTM(units = 10, activation='relu', input_shape = (look_back, 1)))
    # model.add(Dropout(0.2))

    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=[NMSE_metric])

    num_epochs = 30
    model.fit(train_generator, epochs=num_epochs, verbose=1)

    close_data = close_data.reshape((-1))

    num_prediction = 10
    forecast = predict(num_prediction, model)
    forecast_dates = predict_dates(num_prediction)

    sub[code] = forecast

NameError: name 'tqdm' is not defined

In [4]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.90
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

4665
519


In [5]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

def NMSE_metric(y_true, y_pred):
    #nmse = K.mean(K.square(y_true - y_pred)/K.square(y_true), axis=-1)
    nmse = K.square(y_true - y_pred)/K.square(y_true + 1e-8)
    return nmse 

In [10]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 10, activation='relu', input_shape = (look_back, 1)))
# model.add(Dropout(0.2))

# model.add(LSTM(units = 50, activation='relu', return_sequences = True))
# model.add(Dropout(0.2))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[NMSE_metric])

num_epochs = 30
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/100
233/233 [==============================] - 2s 6ms/step - loss: 796000896.0000 - NMSE_metric: 1.8263
Epoch 2/100
233/233 [==============================] - 1s 6ms/step - loss: 65455368.0000 - NMSE_metric: 0.1603
Epoch 3/100
233/233 [==============================] - 1s 6ms/step - loss: 11534887.0000 - NMSE_metric: 0.0229
Epoch 4/100
233/233 [==============================] - 2s 7ms/step - loss: 1129552.3750 - NMSE_metric: 0.0025
Epoch 5/100
233/233 [==============================] - 2s 7ms/step - loss: 199654.2656 - NMSE_metric: 6.7530e-04
Epoch 6/100
233/233 [==============================] - 2s 6ms/step - loss: 168648.9531 - NMSE_metric: 6.1432e-04
Epoch 7/100
126/233 [===============>..............] - ETA: 0s - loss: 173264.5781 - NMSE_metric: 5.8844e-04

In [7]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [8]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 10
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [9]:
trace4 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode='lines',
    name = 'predict'
)

fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

In [1]:
sub = pd.read_csv('data/sample_submission.csv')
sub

NameError: name 'pd' is not defined